# Imports

In [2]:
import time
import serial
import sqlite3
import json
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import argrelextrema

# BiometricSignal Class

In [6]:
class BiometricSignal: 
    """
    A class used to represent an ECG signal

    ...

    Attributes
    ----------
    captured_signal_csv : str
        a string representing a path to a csv file

    Methods
    -------
    capture_signal()
        Captures an ECG signal with an Arduino and AD8232
    output_signal_to_csv()
        Outputs the signal captured to a csv name in 'captured_signal_csv'
    filter_captured_signal()
        Filters the ECG to reduce noise
    amend_signal(filtered_signal)
        Removes start and end of signal which are usually very noisy
    find_r_peaks(amended_signal)
        Finds the R-peaks in the signal
    """
    
    captured_signal_csv = "./assets/subject_raw_ecg.csv"
#     captured_signal_csv = "./subject2.csv"
#     captured_signal_csv = "./subject3a.csv"

    def __init__(self):
        """
        Parameters
        ----------
        captured_ecg : list
            List containing the ECG captured by the Arduino and AD8232
        filtered_signal : numpy.ndarray
            The signal with most of the noise removed
        amended_signal : numpy.ndarray
            The signal with the start and the end removed
        r_peaks : tuple
            Contains the positions of the R-peaks in the signal
        """
        
        self.captured_ecg = []
        self.filtered_signal = np.ndarray([])
        self.amended_signal = np.ndarray([])
        self.r_peaks = ()
        
        
    def capture_signal(self):
        r"""Captures ECG signal from Arduino and AD8232.

        This function captures an ECG signal with an Arduino and AD8232
        on the port specified in `serial_data`. Before it begins capturing,
        there is a small delay (e.g. 2 seconds) to allow the user to get
        comfortable. After this delay it will capture the ECG signal and
        assign it to the list `self.captured_ecg`.
        
        Parameters
        ----------
        serial_data : serial.serialposix.Serial, optinal
            Identifies the port to record the ECG signal, default is
            serial.Serial("/dev/ttyACM2", 9600)
        capture_time : int, optional
            Time in seconds to capture ECG signal for, default is 5 
        start_delay : numpy.ndarray
            Time in seconds to delay before recording starts, default is 2

        """
        
        t_end = time.time() + capture_time
        time.sleep(start_delay)

        while time.time() < t_end:
            while serial_data.inWaiting() == 0:
                pass
            temp_string = serial_data.readline()
            serial_string = (
                str(temp_string)
                .replace("b", "")
                .replace("'", "")
                .replace("\\r", "")
                .replace("\\n", "")
            )
            if len(serial_string) == 3:
                self.captured_ecg.append(int(serial_string))
#         return self.captured_ecg


    def output_signal_to_csv(self):
        
        r"""Coverts `self.captured_ecg` to a csv

        This function converts the list `self.captured_ecg` to a csv and
        saves it to the location set in `captured_signal_csv`.
        
        """
        
        with open(BiometricSignal.captured_signal_csv, "w") as ecg_file:
            ecg_file.write("voltage,\n")
            for item in self.captured_ecg:
                ecg_file.write("%s,\n" % item)
            ecg_file.write("0,")


    def filter_captured_signal(self):
        
        r"""Reduces noise in the signal.

        Reduces noise which in picked up when recording an ECG signal.
        It uses a Butterworth filter to remove jagged parts of the signal
        and smoothen it give a more truer represtation of the ECG signal.
        
        Returns
        -------
        self.filtered_signal : numpy.ndarray
            A filtered version of the captured ECG signal 

        """
            
        numerator, denominator = signal.butter(4, 0.25, analog=False)
        data = pd.read_csv(BiometricSignal.captured_signal_csv)
        sig = data["voltage"]
        self.filtered_signal = signal.filtfilt(numerator, denominator, sig)
        return self.filtered_signal
    
    def amend_signal(self, start = 100, end = -100):
        r"""Removes the begining and end of a filtered signal

        Sometimes there can be a lot of noise at the start and of a ECG
        signal which the `filter_captured_signal` function is unable to remove.
        The noise is caused by connecting to and from the capture device. This
        function takes care of that of that noise by removing the start and end of
        the captured ECG signal.
        
        Parameters
        ----------
        start : int, optinal
            The position to start the clip the ECG signal from, default is 100
        end : int, optional
            The position to end the clip of the ECG signal, default is 100
            
        Returns
        -------
        self.amended_signal : numpy.ndarray
            An ammended version of `filtered_signal` 
            
        """
             
        self.amended_signal = self.filtered_signal[start:end]
        return self.amended_signal
    
    def find_r_peaks(self, threshold = 400):
        r"""Finds the R-peaks in an amended signal

        This function finds the R-peaks in an amended signal. R-peaks are 
        the highest part of a signal so it uses a threshold to illimitate lower
        unnesscary features which may be picked up as a false positive. 
        
        Parameters
        ----------
        threshold : int, optinal
            The position to start looking for R-peaks from, default is 400
            
        Returns
        -------
        self.r_peaks : tuple
            A tuple containing the positions of the R-peaks 
            
        """
        
        no_of_rows = self.amended_signal.shape[0]
        line_numbers = []
        theVoltage = []


        for i in range(0, no_of_rows):
            if self.amended_signal[i] > threshold:
                theVoltage.append(self.amended_signal[i])
            else:
                theVoltage.append(0)
            line_numbers.append(i)    


        ecg_plot = np.concatenate((theVoltage, line_numbers))

        self.r_peaks = argrelextrema(ecg_plot, np.greater, order=5)
        
        return self.r_peaks


# ----------------------------------------------------------------------------------------------
#                                       CODE FOR TESTING
# ----------------------------------------------------------------------------------------------
biosig = BiometricSignal()
biosig.filter_captured_signal()
biosig.amend_signal()
biosig.find_r_peaks()
# print(biosig.filter_captured_signal)
# TEST_filtered_signal = TEST_signal.filter_captured_signal()
# TEST_amended_signal = TEST_signal.amend_signal()
# TEST_r_peaks = TEST_signal.find_r_peaks()
# testytesttest = np.ndarray([])
# serial_data = serial.Serial("/dev/ttyACM2", 9600)
# print(type(TEST_r_peaks))
# help(BiometricSignal)
# plt.figure(num=None, figsize=(10, 5), dpi=80, facecolor="w", edgecolor="k")
# plt.plot(TEST_amended_signal, color="#000000", linewidth=1)
# plt.axis('off')
# plt.savefig("project_presentation.png", dpi=150, quality=100, bbox_inches='tight')
# plt.tight_layout()
# plt.show()

(array([ 110,  244,  373,  499,  623,  748,  873,  998, 1123]),)

# Segment Class

In [8]:
class Segment:
    """
    A segment is a section of an ECG signal between two neighbouring R-peaks

    ...
    
    Attributes
    ----------
    no_of_segments : int
        a int representing the number of segemnts to overlay
        
    Methods
    -------
    combining_segments()
        Combines multiple segments, overlayed on each other
    get_mean_of_segments()
        Finds the mean of multiple segments
    standardise_mean_segment()
        Standardises a mean segment for processing
    find_features_lower()
        Finds the lower unique features of a segment
    find_features_higher()
        Finds the higher unique features of a segment

    """
    
    no_of_segments = 5
    
    
    def __init__(self, bio_signal):
        """
        Parameters
        ----------
        bio_signal : BiometricSignal
            An ECG signal object
        combined_seg : numpy.ndarray
            Contains multiple segments overlayed on each other 
        mean_segment : numpy.ndarray
            The result of combining multiple segments
        standardised_mean_segment : numpy.ndarray
            Standareised mean segment, standisatin in nesscessary for processing
        features_lower() : tuple
            The lower side unique features of a segment
        features_higher() : tuple
            The higher side unique features of a segment
        """
        
        self.bio_signal = bio_signal
        self.combined_seg = self.combining_segments()
        self.mean_segment = self.get_mean_of_segments()
        self.standardised_mean_segment = self.standardise_mean_segment()
        self.features_lower = self.find_features_lower()
        self.features_higher = self.find_features_higher()


    def combining_segments(self):
        r"""Combines multiple segments, overlayed on each other

        This function combines multiple neighbouring R-peaks and overlays them
        to form a single segment. This is the first step in finding a mean
        segment.
                   
        Returns
        -------
        combined_seg : numpy.ndarray
            A numpy array containing multiple segments overlayed on one another
        """
        
        combined_seg_does_not_exist = True
        smallest_seg = None
        
        for i in range (0, self.no_of_segments):
            segment_start = self.bio_signal.r_peaks[0][i]
            segment_end = self.bio_signal.r_peaks[0][i+1]

            extracted_segment = self.bio_signal.amended_signal[segment_start:segment_end]
            if smallest_seg == None:
                smallest_seg = len(extracted_segment)
            elif (len(extracted_segment) < smallest_seg):
                smallest_seg = len(extracted_segment)

            if combined_seg_does_not_exist:
                # adds additional zeros to the end of the first segment
                # this is to prevent a crash as segments may have differnet lengths
                combined_seg = np.zeros(len(extracted_segment) + 100)
                combined_seg_does_not_exist = False
            for j in range(0,len(extracted_segment)):
                combined_seg[j] =  combined_seg[j] + extracted_segment[j]
            
        combined_seg = np.trim_zeros(combined_seg)
        combined_seg = combined_seg[0:smallest_seg]
        return combined_seg
                

    def get_mean_of_segments(self):
        r"""Finds the mean of multiple segments

        This function finds the mean segment by looping through each element in
        the combined segment and dividing it by `no_of_segments`

        mean_segment : numpy.ndarray
            A numpy array containing the mean segment of the combined segments
        """
        
        mean_segment = np.array([])
        
        for k in range (0, len(self.combined_seg)):
            mean_segment = np.append (mean_segment, self.combined_seg[k] / self.no_of_segments)
            
        return mean_segment

    
    def standardise_mean_segment(self):
        r"""Standardises a mean segment for processing

        This function is required because not all ECG signals don't start at zero. The
        lowest value is found and this is subtracted from every element in the
        mean segment.
            
        Returns
        -------
        standardised_mean_segment : numpy.ndarray
            Standareised mean segment, standisatin in nesscessary for processing
        """        
        
        standardised_mean_segment = np.array([])
        
        min_of_mean_segment = min(self.mean_segment)
        for value in range(0, len(self.mean_segment)):
            standardised_mean_segment = np.append (standardised_mean_segment, self.mean_segment[value] - min_of_mean_segment)

        return standardised_mean_segment
    
    def find_features_lower(self):
        r"""Finds the lower unique features of a segment

        This function uses argrelextrema, part of SciPy. Argrelextrema
        calculates the relative extrema of data, this means it examines
        data at either side of a point on the segment to identify variation.
        When if finds broad variations between points, it considers it unique, 
        labels that point as a feature and stores it in a tuple.
            
        Returns
        -------
        features_lower : tuple
            A tuple containing the positions of lower unique features. 
        """
        
        features_lower = argrelextrema(self.mean_segment, np.less, order=5)
        self.features_lower = (features_lower[0],self.mean_segment[features_lower[0]])       
        return self.features_lower


    def find_features_higher(self):
        r"""Finds the higher unique features of a segment

        This function uses argrelextrema, part of SciPy. Argrelextrema
        calculates the relative extrema of data, this means it examines
        data at either side of a point on the segment to identify variation.
        When if finds broad variations between points, it considers it unique, 
        labels that point as a feature and stores it in a tuple.
            
        Returns
        -------
        features_higher : tuple
            A tuple containing the positions of higher unique features. 
        """
        
        features_higher = argrelextrema(self.mean_segment, np.greater, order=5)
        self.features_higher = (features_higher[0],self.mean_segment[features_higher[0]]) 
        return self.features_higher

        
# ----------------------------------------------------------------------------------------------
#                                       CODE FOR TESTING
# ----------------------------------------------------------------------------------------------
seg = Segment(biosig)
seg.combining_segments()
seg.
seg.
seg.
# print(type(TEST_segment.features_lower))
# print(TEST_segment.combined_seg)
# TEST_combined_seg = TEST_segment.combining_segments()
# TEST_mean_segment = TEST_segment.get_mean_of_segments(TEST_combined_seg)
# TEST_standardised_mean_segment = TEST_segment.standardise_mean_segment(TEST_mean_segment)

# END OF SEGMENT CLASS

array([2548.97502232, 2458.50363072, 2188.71486457, 1833.36450487,
       1513.84803125, 1321.31750675, 1280.24088199, 1351.16072336,
       1464.07686936, 1557.73497385, 1602.98761294, 1603.46827222,
       1581.00305249, 1558.37530351, 1548.63905135, 1553.36720097,
       1566.95026276, 1582.36964276, 1595.10454458, 1604.14778807,
       1610.91113603, 1617.45704888, 1625.19842692, 1634.49869171,
       1644.97716774, 1656.05755783, 1667.38446742, 1678.96289291,
       1691.05219628, 1703.92702926, 1717.64269042, 1731.92263723,
       1746.21370678, 1759.85901633, 1772.27503956, 1783.0278814 ,
       1791.78877457, 1798.25649021, 1802.1610223 , 1803.36494664,
       1801.95726796, 1798.22496327, 1792.49163213, 1784.91852134,
       1775.39447319, 1763.60224264, 1749.25551581, 1732.37918865,
       1713.4448657 , 1693.26463566, 1672.73245751, 1652.60424507,
       1633.43551962, 1615.64925195, 1599.62879227, 1585.7533044 ,
       1574.36006538, 1565.66934825, 1559.71239899, 1556.28300

# Templates Class

In [53]:
class Templates:
    """
    A class for templates for ECG signals

    ...

    Methods
    -------
    create_table()
        Creates the templates table in the database if it doesn't already exist
    add_entry_to_database(features_lower, features_higher, standardised_mean_segment, x_length, y_length)
        Adds a record to the database
    convert_features_to_percentages(features, length)
        Converts the feature locations to percentages
    query_db(theName)
        Searchs for a name in the database
    does_user_exist(theName)
        Checks to see if user already exists in the database
    get_template()
        Retrieves a template from the database
    template_matching()
        Compares the template in the database with a captured one
    get_start_end_locations(self, feature_x, feature_y, image_width, image_height)
        Uses a feature to determine a area to check for a match
    get_feature_location(self, x, y, image_width, image_height)
        ddddd
    get_feature_from_template(self, db_template_ecg, x_start, x_end, y_start, y_end)
        sssssss
    """

    def __init__(self, name):
        """
        Parameters
        ----------
        name : str
            The name of the user
        conn : sqlite3.Connection
            Connection to the SQLite3 database
        c : sqlite3.Cursor
            Connection for the SQLite3 database
        standardised_mean_segment : numpy.ndarray
            Standareised mean segment, standisatin in nesscessary for processing
        featuresX : list
            A list of all the features on the X axis
        featuresY : list
            A list of all the features on the Y axis
        """

        self.name = name
        self.conn = sqlite3.connect('templatesDB.db')
        self.c = self.conn.cursor()
        self.create_table()
        self.standardised_mean_segment = []
        self.featuresX = []
        self.featuresY = []
    
    def create_table(self):
        r"""Creates the templates table in the database if it doesn't already exist

        Checks to see if table for the templates exists and creates it if it doesn't.
        The table contains the name, mean_segment, features along the X axis and 
        features along the Y axis
        """
        self.c.execute("""CREATE TABLE IF NOT EXISTS templates 
                                            ( name text,
                                                mean_segment text,
                                                featuresX text,
                                                featuresY text)""")
        self.conn.commit()
    
    
    def add_entry_to_database(self, features_lower, features_higher, standardised_mean_segment,
                          x_length, y_length):
        r"""Adds a record to the database

        Adds a new user to the database. It starts by converting the feature locations
        to percentages for both the lower and higher features. This standardises the 
        locations making matching more accurate. It then adds them to the database,
        combining X axises of both lower and higher features, and doing the
        same for the Y axises. It also sotres the user and the user's standarised
        mean segment.
        
        Parameters
        ----------
        features_lower : tuple
            A tuple containing the positions of lower unique features.
        features_higher : tuple
            A tuple containing the positions of higher unique features.
        standardised_mean_segment : numpy.ndarray
            Standareised mean segment, standisatin in nesscessary for processing
        x_length : int
            The length of the X axis
        y_length : int
            The length of the Y axis 
        """
        if self.does_user_exist(self.name) != True:
            thefeaturesLowerX = self.convert_features_to_percentages(features_lower[0].tolist(), x_length)
            thefeaturesLowerY = self.convert_features_to_percentages(features_lower[1].tolist(), y_length)
            thefeaturesHigherX = self.convert_features_to_percentages(features_higher[0].tolist(), x_length)
            thefeaturesHigherY = self.convert_features_to_percentages(features_higher[1].tolist(), y_length)

            self.c.execute("INSERT INTO templates VALUES (:name, :mean_segment, :featuresX, :featuresY)",
                                                {'name': self.name,
                                                 'mean_segment': json.dumps(standardised_mean_segment.tolist()),
                                                 'featuresX': json.dumps(thefeaturesLowerX + thefeaturesHigherX),
                                                 'featuresY': json.dumps(thefeaturesLowerY + thefeaturesHigherY)})


            self.conn.commit()
        
        
    def convert_features_to_percentages(self, features, length):
        r"""Converts a feature's location to a percentage

        Converts the location of a feature to a percentage for
        standardisaion. Standardisation helps with template matching.
        
        Parameters
        ----------
        features : numpy.ndarray
            description
        length : int
            The length of the features
            
        Returns
        -------
        the_row : list
            Returns a list of ints
            
        """
        
        the_row = []

        for value in features:
            pos_percentage = (value * 100) // length
            the_row.append(pos_percentage)
            
        return the_row
    

    def query_db(self, theName):
        r"""Queries the database with the name provided in arguments

        Queries the database with the name provided in arguments and
        returns the result if there is one.
        
        Parameters
        ----------
        theName : str
            The name used in the query
            
        Returns
        -------
        query_response : tuple
            The result, if any, of the query
            
        """
        
        self.c.execute("SELECT * FROM templates WHERE name IS (:name)", {'name': theName})
        query_response=(self.c.fetchone())        
        return query_response
    
    def does_user_exist(self, theName):
        r"""Queries the database with the name provided in arguments

        Queries the database with the name provided in arguments and
        returns a bool.
        
        Parameters
        ----------
        theName : str
            The name used in the query
            
        Returns
        -------
        bool
            True or false, depending on if name is found in database
        """
        
        if self.query_db(theName) == []:
            return False
        else:
            return True
        
        
    def get_template(self, theName):
        r"""Retrieves template from the database

        Queries the database with the name provided in arguments and
        returns the result if there is one.
        
        Parameters
        ----------
        theName : str
            The name used in the query
            
        Returns
        -------
        standardised_mean_segment : list
            The users segment
        featuresX : list
            The X cordinates of the the features
        featuresY : list
            The X cordinates of the the features            
        """
        
        if self.does_user_exist(theName):
            query_response = self.query_db(theName)
            self.standardised_mean_segment = json.loads(query_response[1])
            self.featuresX = json.loads(query_response[2])
            self.featuresY = json.loads(query_response[3])
            return self.standardised_mean_segment, self.featuresX, self.featuresY
    
    
    def template_matching(self):
        r"""Compares the captured template with the one in the database

        Compares the captured template with the one in the database. Loops
        through each each feature of the captured ECG signal and compares
        it to that in the database.
            
        Returns
        -------
        success_rate : float
            The success rate of the matches
        """
        
        self.featuresY =  [int(i) for i in featuresY]
     
        db_template_ecg = cv2.imread("./assets/db_template.png", 0)
        captured_ecg = cv2.imread("./assets/TEST_standardised_mean_segment.png", 0)

        
        total_matches = 0
        successful_matches = 0
        
        for x,y in zip(self.featuresX, self.featuresY):
            
            if x > 10 and x < 90 and y > 10 and y < 90 :

                image_width, image_height = db_template_ecg.shape        
                feature_x, feature_y = self.get_feature_location(x, y, image_width, image_height)
                x_start, x_end, y_start, y_end = self.get_start_end_locations(feature_x, feature_y, image_width, image_height)

                feature_to_match = self.get_feature_from_template(db_template_ecg, x_start, x_end, y_start, y_end)

                w, h = feature_to_match.shape[::-1]
                res = cv2.matchTemplate(captured_ecg,feature_to_match,cv2.TM_CCOEFF_NORMED)

                threshold = 0.9      
                loc = np.where( res >= threshold)
                if np.amax(res) > threshold:
                    total_matches += 1
                    successful_matches += 1
                else:
                    total_matches += 1

        success_rate = (successful_matches * 100) / total_matches
        return success_rate
    

    
    def get_start_end_locations(self, feature_x, feature_y, image_width, image_height):
        r"""Identifies area to check on the template

        The features are a single point on the segment, this function
        takes the area around that point to include in the template match
        to increase its accuracity. It also takes into account where
        the point is, and if its close to an edge to make sure it doesn't
        go out of bounds.
        
        Parameters
        ----------
        feature_x : list
            The cordinates on X axis
        feature_y : list
            The cordinates on Y axis
        image_width : int
            The segments width
        image_height : int
            The segments length
            
        Returns
        -------
        x_start : int
            The starting X cordinate needed for OpenCV
        x_end : int
            The ending X cordinate needed for OpenCV
        y_start : int
            The starting Y cordinate needed for OpenCV
        y_end : type
            The ending Y cordinate needed for OpenCV  
        """
        
        margin = 10
        
        if feature_x < margin:
            x_start = feature_x
        else:
            x_start = feature_x - margin

        if feature_x > image_width:
            x_end = image_width
        else:
            x_end = feature_x + margin  

        if feature_y < margin:
            y_start = 0
        else:
            y_start = feature_y - margin

        if feature_y > image_height:
            y_end = image_height
        else:
            y_end = feature_y + margin
            
        return x_start, x_end, y_start, y_end
            
            
    def get_feature_location(self, x, y, image_width, image_height):
        r"""Converts a feature location to a percentage

        Converts a feature location to a percentage for standardisaion.
        Standardisation makes template matching more accurate.
        
        Parameters
        ----------
        x : int
            description
        y : int
            description
        image_width : int
            description
        image_height : int
            description
            
        Returns
        -------
        feature_x : int
            description
        feature_y : int
            description
        """
        
        feature_x = (image_width // 100) * x
        feature_y = (image_height // 100) * y
        return feature_x, feature_y
    
    def get_feature_from_template(self, db_template_ecg, x_start, x_end, y_start, y_end):
        r"""Gets feature from template to check

        Converts a feature location to a percentage for standardisaion.
        Standardisation makes template matching more accurate.
        
        Parameters
        ----------
        name : type
            description
            
        Returns
        -------
        name : type
            description
            
        """
        
        feature_to_match = db_template_ecg[x_start:y_start,x_end:y_end]
        return feature_to_match
    
    
#     def show_image(self, captured_ecg):
#         cv2.imshow('Detected',captured_ecg)
#         cv2.waitKey(0) & 0xFF
#         cv2.destroyAllWindows()
    
TEST_templates = Templates('Sam')
TEST_the_name = 'Sam'
TEST_templates.add_entry_to_database(TEST_segment.features_lower, TEST_segment.features_higher,
                                     TEST_segment.standardised_mean_segment, len(TEST_segment.combined_seg), 
                                     len(TEST_segment.standardised_mean_segment))
TEST_query_response = TEST_templates.query_db('Sam')
TestyMacTestFace = json.loads(TEST_query_response[1])

# print(TEST_query_response[1])
# print(TEST_templates.name)
# print(TEST_templates.featuresX)
# print(TEST_templates.featuresY)
# TEST_templates.get_template()
# print(type(TEST_query_response))
# print(TEST_templates.featuresY)
TEST_standardised_mean_segment, TEST_featuresX, TEST_featuresY = TEST_templates.get_template(TEST_the_name)
# print(TEST_standardised_mean_segment)
print(type(TEST_featuresX))
# TEST_templates.get_template()
# TEST_templates.template_matching(TEST_featuresLowerX, TEST_featuresLowerY, TEST_featuresHigherX, TEST_featuresHigherY)


<class 'list'>
